In [1]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


In [2]:
### Set up Yolo UDF
cursor.query("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
    """).df() 

,0
0,"UDF Yolo already exists, nothing added."


In [3]:
### Set up Monodepth UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS MonodepthDetection
            IMPL'/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

,0
0,"UDF MonodepthDetection already exists, nothing..."


In [ ]:
### Set up location UDF
cursor.query(""" 
            CREATE UDF MonodepthDetection
            IMPL'/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

In [ ]:
### Set up intersection query UDF

In [1]:
### Load video
video_name = "boston-seaport-scene-0757-CAM_FRONT.mp4"
scene = "scene-0757"
video_path = "/data/processed/full-dataset/mini/videos/"
cursor.query("DROP TABLE IF EXISTS ObjectDetectionVideos;").df()
cursor.load(file_regex=video_path + video_name, format="VIDEO", table_name='ObjectDetectionVideos').df()

NameError: name 'cursor' is not defined

In [11]:
### Add Camera Configs
%cd ..
from apperception.database import database
result = database.execute(f"SELECT cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic FROM Cameras WHERE cameraId = '{scene}'")

for r in result:
    camera_config = {
         "cameraId": r[0],
         "frameNum": r[1],
         "cameraTranslation": r[2],
         "cameraRotation": r[3],
         "cameraIntrinsic": r[4]
     }
    

/


[('scene-0757',
  1,
  <Point POINT(307.6273244693186 669.5056617266097 1)>,
  [-0.58233327, -0.37710914, -0.61586994, -0.37333748],
  [[1252.8131, 0.0, 826.58813], [0.0, 1252.8131, 469.98465], [0.0, 0.0, 1.0]]),
 ('scene-0757',
  2,
  <Point POINT(307.62887572825804 669.5058308426003 1)>,
  [-0.58236444, -0.37700972, -0.6158898, -0.37335646],
  [[1252.8131, 0.0, 826.58813], [0.0, 1252.8131, 469.98465], [0.0, 0.0, 1.0]]),
 ('scene-0757',
  3,
  <Point POINT(307.62913264675615 669.506507995929 1)>,
  [-0.58237135, -0.37695903, -0.61591053, -0.37336275],
  [[1252.8131, 0.0, 826.58813], [0.0, 1252.8131, 469.98465], [0.0, 0.0, 1.0]]),
 ('scene-0757',
  4,
  <Point POINT(307.62999528129836 669.5087524629632 1)>,
  [-0.58247054, -0.37664166, -0.61586624, -0.3736013],
  [[1252.8131, 0.0, 826.58813], [0.0, 1252.8131, 469.98465], [0.0, 0.0, 1.0]]),
 ('scene-0757',
  5,
  <Point POINT(307.6298703153587 669.5088518241006 1)>,
  [-0.5824467, -0.3766154, -0.61588967, -0.37362626],
  [[1252.8131, 0.

In [8]:
cursor.query("SELECT id, Yolo(data) FROM ObjectDetectionVideos").df()

,objectdetectionvideos.id,yolo.labels,yolo.bboxes,yolo.scores
0,0,"[traffic light, traffic light]","[[1209.5908203125, 363.62115478515625, 1228.27...","[0.44, 0.42]"
1,1,"[truck, traffic light, traffic light, car]","[[1254.342529296875, 425.235595703125, 1471.82...","[0.56, 0.55, 0.35, 0.31]"
2,2,"[truck, traffic light]","[[1262.8585205078125, 426.02020263671875, 1470...","[0.48, 0.35]"
3,3,"[truck, traffic light]","[[1277.282958984375, 429.562744140625, 1503.24...","[0.56, 0.44]"
4,4,"[truck, traffic light, traffic light]","[[1275.486083984375, 423.3941650390625, 1532.6...","[0.67, 0.58, 0.39]"
...,...,...,...,...
232,232,"[car, car, traffic light, truck, traffic light...","[[740.5250244140625, 414.62310791015625, 1099....","[0.88, 0.83, 0.69, 0.6, 0.59, 0.51, 0.46]"
233,233,"[car, car, traffic light, traffic light, person]","[[768.5474853515625, 416.8846130371094, 1049.6...","[0.89, 0.85, 0.66, 0.58, 0.47]"
234,234,"[car, traffic light, car, traffic light, truck]","[[514.0872192382812, 429.6484375, 904.81982421...","[0.76, 0.67, 0.66, 0.59, 0.45]"
235,235,"[car, traffic light, traffic light, truck, per...","[[576.6544189453125, 428.54644775390625, 969.3...","[0.72, 0.69, 0.61, 0.52, 0.41]"


In [6]:
### Run query
cursor.query("""
            SELECT id, Yolo(data) FROM ObjectDetectionVideos
            WHERE Intersection(Location(Yolo(data)., MonodepthDetection(data)))
""").df()


UnexpectedToken: Unexpected token Token('COMMA', ',') at line 3, column 52.
Expected one of: 
	* ID
	* REVERSE_QUOTE_ID
Previous tokens: [Token('DOT', '.')]
